In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import datetime

import wire_detection_utils as wdu

In [ ]:
expansion_size = 2
line_threshold = 100
pixel_binning_size = 3
low_canny_threshold = 100
high_canny_threshold = 200
bin_avg_threshold_multiplier = 0.5

# Creat e a WireDetector instance
wire_detector = wdu.WireDetector(line_threshold, expansion_size, low_canny_threshold, high_canny_threshold, pixel_binning_size, bin_avg_threshold_multiplier)

img = cv2.imread('input_imgs/outdoor_2.png')
assert img is not None, "Image not found"
print(f'Image shape: {img.shape}')

In [ ]:
cv2.imwrite('output_imgs/reference.jpg', img)

# Create segmentation mask
seg_mask = wire_detector.create_seg_mask(img)
cv2.imwrite('output_imgs/seg_mask.jpg', seg_mask)

# Detect wires using the segmentation mask
cartesian_lines, center_line, center_line_midpoint, avg_angle, seg_coords = wire_detector.get_hough_lines(seg_mask)
assert cartesian_lines is not None, "No lines found"

wire_lines, wire_midpoints = wire_detector.get_line_instance_locations(cartesian_lines, center_line, center_line_midpoint, avg_angle, seg_coords)

# Visualize detected wires
hough_img = img.copy()
for x0, y0, x1, y1 in cartesian_lines:
    rand_color = tuple(np.random.randint(0, 256, 3).tolist())
    cv2.line(hough_img, (x0, y0), (x1, y1), rand_color, 1)
cv2.imwrite('output_imgs/hough_detect.jpg', hough_img)

centerline_img = img.copy()
cv2.line(centerline_img, (center_line[0], center_line[1]), (center_line[2], center_line[3]), (0, 255, 0), 1)
cv2.circle(centerline_img, (int(center_line_midpoint[0]), int(center_line_midpoint[1])), 5, (0, 255, 0), -1)
cv2.imwrite('output_imgs/centerline.jpg', centerline_img)

clustered_img = img.copy()
rand_colors = np.random.randint(0, 256, (len(wire_midpoints), 3)).tolist()
for i, (x0,y0,x1,y1) in enumerate(wire_lines):
    cv2.line(clustered_img, (x0,y0), (x1,y1), tuple(rand_colors[i]), 1)

for i, (x0,y0) in enumerate(wire_midpoints):
    cv2.circle(clustered_img, (int(x0), int(y0)), 5, tuple(rand_colors[i]), -1)

cv2.imwrite('output_imgs/cluster_detect.jpg', clustered_img)


Image shape: (270, 480, 3)


True